In [1]:
from RNN import MDNRNN
import torch
import numpy as np 
from torch import nn
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cpu


In [2]:
################
#  DATALOADING #
################

In [3]:

def sliding_windows(data,seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[(i+1):(i+1)+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)



In [15]:
bsz = 200

data = np.load('latent-action.npy')
print(data[:,23233,:])
#Reshaping data and seperating training and test set
sc = MinMaxScaler()
data = sc.fit_transform(data.squeeze(axis = 0))
print(data[23233,:])
seq_length = 4
x, y = sliding_windows(data, seq_length)



train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])).float())
trainY = Variable(torch.Tensor(np.array(y[0:train_size])).float())


testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])).float())
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])).float())


trainingset =[]
for i in range(len(trainY)):
    trainingset.append((trainX[i,:,:], trainY[i]))
    


trainloader = torch.utils.data.DataLoader(trainingset, batch_size=bsz, shuffle=False)





[[ 3.41547799  0.00394334  2.49205041 -2.55889511  0.31533423  1.64574075
  -0.11483234 -2.42293024  0.298       0.452       0.015       0.395
   0.293       0.403       0.311       0.328      -0.298       1.        ]]
[0.95917525 0.40288215 0.97879683 0.27208062 0.40316703 0.67172941
 0.49172997 0.08352195 0.34651163 0.5255814  0.01744186 0.45930233
 0.34069767 0.46860465 0.36162791 0.38139535 0.27456258 0.5       ]


In [16]:

learning_rate = 1e-7
input_size = 18
hidden_size = 256
num_layers = 1
num_classes = 1
gaussians = 5

mdnrnn = MDNRNN(z_size = input_size, n_hidden = hidden_size, n_gaussians = gaussians, n_layers = num_layers).to(device)


def detach(states):
    return [state.detach() for state in states]

In [17]:
optimizer = torch.optim.Adam(mdnrnn.parameters(), lr = learning_rate)

epochs = 500
random = torch.zeros(200,4,17)

for epoch in range(epochs):
    # Set initial hidden and cell states
    hidden = mdnrnn.init_hidden(bsz)

    # Get mini-batch inputs and targets
    total_loss = 0
    j = 0
    for i, (inputs, targets) in enumerate(trainloader):
        # Forward pass
        if inputs.size(0) != bsz:
            continue
        hidden = detach(hidden)
        (pi, mu, sigma), hidden = mdnrnn(inputs.to(device), hidden)

        loss = mdnrnn.criterion(targets.to(device), pi, mu, sigma)
        # Backward and optimize
        mdnrnn.zero_grad()
        total_loss += loss.item()

        loss.backward()
        clip_grad_norm_(mdnrnn.parameters(), 0.5)
        optimizer.step()
        j += 1
    total_loss /=j
    if epoch % 2 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, epochs, total_loss))
        torch.save(mdnrnn.state_dict(), '../Models/transition-regression-new')




Epoch [0/500], Loss: 1.0646
Epoch [2/500], Loss: 1.0472
Epoch [4/500], Loss: 1.0258
Epoch [6/500], Loss: 0.9953
Epoch [8/500], Loss: 0.9451
Epoch [10/500], Loss: 0.8397
Epoch [12/500], Loss: 0.5275
Epoch [14/500], Loss: 0.2591
Epoch [16/500], Loss: 0.0644
Epoch [18/500], Loss: -0.1056
Epoch [20/500], Loss: -0.2296
Epoch [22/500], Loss: -0.3104
Epoch [24/500], Loss: -0.3616
Epoch [26/500], Loss: -0.3968
Epoch [28/500], Loss: -0.4245
Epoch [30/500], Loss: -0.4492
Epoch [32/500], Loss: -0.4731
Epoch [34/500], Loss: -0.4968
Epoch [36/500], Loss: -0.5202
Epoch [38/500], Loss: -0.5430
Epoch [40/500], Loss: -0.5650
Epoch [42/500], Loss: -0.5861
Epoch [44/500], Loss: -0.6062
Epoch [46/500], Loss: -0.6254
Epoch [48/500], Loss: -0.6436
Epoch [50/500], Loss: -0.6609
Epoch [52/500], Loss: -0.6773
Epoch [54/500], Loss: -0.6930
Epoch [56/500], Loss: -0.7079
Epoch [58/500], Loss: -0.7222
Epoch [60/500], Loss: -0.7359
Epoch [62/500], Loss: -0.7490
Epoch [64/500], Loss: -0.7616
Epoch [66/500], Loss: -0

In [50]:
mdnrnn.load_state_dict(torch.load('../Models/transition-regression-new', map_location='cpu'))

mdnrnn.eval()
size = train_size+test_size
zero = np.random.randint(testX.size(0))

one = np.random.randint(testX.size(1))


x = testX[zero:zero+1, one:one+1, :]
y = testX[zero:zero+1, one+1:one+2, :]

hidden = mdnrnn.init_hidden(1)
(pi, mu, sigma), _ = mdnrnn(x, hidden)

y_preds = [torch.normal(mu, sigma)[:, :, i, :] for i in range(5)]

new = [x.detach().numpy().reshape(1,18) for tens in y_preds]

print(new)

data_predict = sc.inverse_transform(new[0].reshape(1,-1))
dataY_truth = sc.inverse_transform(y.flatten().reshape(1,))
print(data_predict[0:5])
print("---------------------------------------")
print(data_truth[0:5])

[array([[0.697392  , 0.7311963 , 0.78749067, 0.34954572, 0.22417575,
        0.81833375, 0.44740587, 0.26787177, 0.2767442 , 0.4616279 ,
        0.29651162, 0.6662791 , 0.6465116 , 0.3604651 , 0.38139534,
        0.6488372 , 0.31493944, 0.        ]], dtype=float32), array([[0.697392  , 0.7311963 , 0.78749067, 0.34954572, 0.22417575,
        0.81833375, 0.44740587, 0.26787177, 0.2767442 , 0.4616279 ,
        0.29651162, 0.6662791 , 0.6465116 , 0.3604651 , 0.38139534,
        0.6488372 , 0.31493944, 0.        ]], dtype=float32), array([[0.697392  , 0.7311963 , 0.78749067, 0.34954572, 0.22417575,
        0.81833375, 0.44740587, 0.26787177, 0.2767442 , 0.4616279 ,
        0.29651162, 0.6662791 , 0.6465116 , 0.3604651 , 0.38139534,
        0.6488372 , 0.31493944, 0.        ]], dtype=float32), array([[0.697392  , 0.7311963 , 0.78749067, 0.34954572, 0.22417575,
        0.81833375, 0.44740587, 0.26787177, 0.2767442 , 0.4616279 ,
        0.29651162, 0.6662791 , 0.6465116 , 0.3604651 , 0.3813953

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
torch.save(mdnrnn.state_dict(), '../Models/transition-regression')

In [ ]:
import pickle
with open('sc_midprice', 'wb') as f:
    pickle.dump(sc_y, f)
    